# How to Train a LEOPARD

This notebook provides a brief instruction for training your own LEOPARD. You can also use the following code to reproduce the result of the MGH COVID dataset reported in our paper. The result may vary slightly with each run due to the stochastic mechanisms involved. Any questions regarding the code, please contact the zookeeper: Siyu Han (siyu.han@tum.de)

## Step 1: Import required modules

Before proceeding, ensure that the following dependencies are properly installed:
- python: 3.79
- numpy: 1.21.5
- pandas: 1.3.5
- scikit-learn: 1.0.2
- pytorch: 1.11.0
- pytorch_lightning: 1.6.4
- tensorboard: 2.10.0
- cuda (if use GPU): 11.3

*The listed version numbers represent those utilized during our development. We cannot guarantee the compatibility or identical outcomes with different versions.*

In [1]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from src.data import *
from src.train import TrainLEOPARD

## Step 2: Prepare dataset

This is done by the function `prepare_dataset()`. This function does the following things: 
1. load data and format them into training/validation/test sets with the function `load_split_data()`. 
2. scale data use a scaler with the function `scale_data()`. 
3. create an instance of `OmicsDataset` class using scaled data for each data split.

The function `prepare_dataset()` receives arguments for the following parameters:
- `data_dir`: a string to specify the folder containing the data of two views (A and B) and two timepoints (1 and 2). The data are saved in .csv files with names like "vA_t1_test.csv", "vB_t1_train.csv". "v" and "t" denote views and timepoints. "v\*_t\*_train.csv" used for model training and validation. Missing values can be encoded as NA. Even if data from view B at timepoint 2 are completely missing, you still need to provide a "vB_t2_train.csv" file with the corresponding sample ID and variable ID, and missing values are indicated with NA. "v\*_t\*_test.csv" is optional and only used for performance evaluation. Default: `"data\MGH_COVID"`
- `valSet_ratio`: a numeric value between 0 and 1 to specify the ratio of data from "v\*_t\*_train.csv" used for constructing the validation set. Default: `0.2`
- `trainNum`: a numeric value or `"all"` indicating how many samples will be randomly selected from the training data for training. Default: `"all"`
- `obsNum`: a numeric value or `"all"` indicating how many samples from "vB_t2_train.csv" will be used for training. Default: `0` 
- `use_scaler`: a string to indicate which scaler is used to scale data. Support `"standard"`, `"robust"`, and `"minmax"`. Description of the scalers please refer to the User Guide of `sklearn.preprocessing`. Default: `"standard"`
- `set_seed`: a numeric value to set seed for reproducible results. Default: `1`
- `save_data_dir`: `None` or a path used for saving the indices of samples used in training. Default: ``

In [2]:
obsNum = 0
trainNum = "all"

loaded_data = prepare_dataset(data_dir="data/MGH_COVID", valSet_ratio=0.2, 
                              trainNum=trainNum, obsNum=obsNum, 
                              use_scaler="standard", set_seed=1, save_data_dir=None)

+ loading data:
  -load vA_t1 as vA_t1
  -load vA_t2 as vA_t2
  -load vB_t1 as vB_t1
  -load vB_t2 as vB_t2
+ Data scaling using standard scaler:
  - vA_t1: use scaler_vA
  - vA_t2: use scaler_vA
  - vB_t1: use scaler_vB
  - vB_t2: use scaler_vB


## Step 3: Create an instance of the `TrainLEOPARD` class

The pytorch code of LEOPARD is organized into `TrainLEOPARD`, a `LightningModule`. LEOPARD is fully customizable. You can adapt your LEOPARD using the following parameters when instantiating an instance of the `TrainLEOPARD` class:
- `train_set`, `val_set`, `test_set`: data prepared by `prepare_dataset()` for training, validation and test
- `scaler_viewA`, `scaler_viewB`: scaler used by `prepare_dataset()` to transform data in two views
- `pre_layers_viewA`, `pre_layers_viewB`: pre-layers for view A and view B to convert them to the same dimension. Default: `[64]`, `[64]`
- `post_layers_viewA`, `post_layers_viewB`: post-layers for view A and view B to convert embeddings back to data in the original dimension. Default: `[64]`, `[64]`

- `encoder_content_layers`: layers for the content encoder. A list where the length indicates the total number of layers, and each element specifies the size of the corresponding layer. Default: `[64, 64, 64]`
- `encoder_content_norm`: a list indicates if using normalization for the layers in the content encoder. Supported `"instance"`, `"batch"`, and `"none"`. Default: `["instance", "instance", "instance"]`
- `encoder_content_dropout`: a list specifies dropout rate for each layer in the content encoder. Default: `[0, 0, 0]`
    
- `encoder_temporal_layers`: layers for the temporal encoder. Default: `[64, 64, 64]`
- `encoder_temporal_norm`: if use normalization for the layers in the temporal encoder? Supported `"instance"`, `"batch"`, and `"none"`. Default: `["none", "none", "none"]`
- `encoder_temporal_dropout`: dropout rate for each layer in the temporal encoder. Default: `[0, 0, 0]`
    
- `generator_block_num`: how many layers/blocks are used for the generator. Default: `3`
- `generator_norm`: if use normalization for the layers in the generator? Supported `"instance"`, `"batch"`, and `"none"`. Default: `["none", "none", "none"]`
- `generator_dropout`: dropout rate for each layer in the generator. Default: `[0, 0, 0]`
- `merge_mode`: re-entangle content and temporal representations by concatenation (`"concat"`) or AdaIN (`"adain"`)? Default: `"adain"`
    
- `discriminator_layers`: layers for the multi-task discriminator. Default: `[128, 128]`
- `discriminator_norm`: if use normalization for the layers in the discriminator? Supported `"instance"`, `"batch"`, and `"none"`. Default: `["none", "none"]`
- `discriminator_dropout`: dropout rate for each layer in the discriminator. Default: `[0, 0]`
    
- `reconstruction_loss`: use `"MSE"` or `"MAE"` to compute reconstruction loss? Default: `"MSE"`
- `adversarial_loss`: use `"MSE"` or `"BCE"` to compute adversarial loss? Default: `"MSE"`
- `weight_reconstruction`, `weight_adversarial`, `weight_representation`, `weight_contrastive`: weights for different losses. Default: `1`, `1`, `0.1`, `0.1`
- `contrastive_temperature`: temperature for NT-Xent-based contrastive loss. Default: `0.05`
    
- `lr_G`, `lr_D`: learning rate for generator process (encoders and generator) and discrimination process (discriminator). ***You need to tune this for your own datasets.*** Default: `0.0005`, `0.0005`
- `b1_G`, `b1_D`: beta_1 for Adam Optimizer. Default: `0.9`, `0.9`
- `lr_scheduler_G`, `lr_scheduler_D`: `"none"` or use `"LambdaLR"` or `"SGDR"` as lr scheduler? Default: `"none"`, `"none"`
    
- `use_projection_head`: if use projection head for contrastive learning? Default: `False`
- `projection_output_size`: set output size of projection head. Ignored if `use_projection_head=False`.  Default: `0`
- `batch_size`: batch size. ***You need to adjust this based on your sample size.*** Default: `32`
- `note`: add some additional texts as a hyperparameter to label each run.  Default: `"obsNum_" + str(obsNum)`

*Some hyperparameters (especially `lr_G`, `lr_D`, and `batch_size`) may need to be tuned for your own datasets.*

In [3]:
my_leopard = TrainLEOPARD(train_set=loaded_data['train_set'], val_set=loaded_data['val_set'],
                          test_set=loaded_data['test_set'],
                          scaler_viewA=loaded_data['scaler_viewA'], scaler_viewB=loaded_data['scaler_viewB'],
                          
                          pre_layers_viewA=[64], pre_layers_viewB=[64],
                          post_layers_viewA=[64], post_layers_viewB=[64],
                          
                          encoder_content_layers=[64, 64, 64],
                          encoder_content_norm=['instance', 'instance', 'instance'],
                          encoder_content_dropout=[0, 0, 0],
                          
                          encoder_temporal_layers=[64, 64, 64],
                          encoder_temporal_norm=['none', 'none', 'none'],
                          encoder_temporal_dropout=[0, 0, 0],
                          
                          generator_block_num=3,
                          generator_norm=['none', 'none', 'none'],
                          generator_dropout=[0, 0, 0],
                          merge_mode='adain',
                          
                          discriminator_layers=[64, 64],
                          discriminator_norm=['none', 'none'],
                          discriminator_dropout=[0, 0],
                          
                          reconstruction_loss='MSE', adversarial_loss='MSE',
                          weight_reconstruction=1, weight_adversarial=1,
                          weight_representation=0.1,
                          weight_contrastive=0.1,
                          contrastive_temperature=0.05,
                          
                          lr_G=0.0005, lr_D=0.0005, b1_G=0.9, b1_D=0.9,
                          lr_scheduler_G='none', lr_scheduler_D='none',
                          
                          use_projection_head=False, projection_output_size=0,
                          batch_size=32, note="obsNum_" + str(obsNum))

## Step 4: Create an instance of the `Trainer` class

This is done by calling `Trainer()` from `pytorch_lightning`. `Trainer` can help you train your LEOPARD. Here we use the following settings (please refer to its Docs for a comprehensive parameter explanation):
- `enable_progress_bar`: show progress bar? Default: `True`
- `log_every_n_steps`: a numeric value that specifies the interval, in steps, at which metrics should be logged. Default: `3`
- `max_epochs`: a numeric value that defines the maximum number of epochs the training loop should run. Default: `100`
- `gpus`: a value indicating which GPUs to use. Default: `1 if torch.cuda.is_available() else None`
- `logger`: a tensorboard logger responsible for logging training/validation metrics and other experiment details.

In [4]:
save_dir = os.path.join("lightning_logs", "trainNum_" + str(trainNum))
name = "obsNum_" + str(obsNum)

trainer = pl.Trainer(
    enable_progress_bar=False,
    log_every_n_steps=3,
    max_epochs=199,
    gpus=1 if torch.cuda.is_available() else None,
    logger=TensorBoardLogger(save_dir=save_dir, name=name)
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Step 5: Train your LEOPARD

Now let's train your LEOPARD!

Optional: you can also visualize the training process with the logger. Use the `%tensorboard` magic command or call it in command line: `tensorboard --logdir *save_dir* --port 8080`
(*use your own `save_dir` and port number*)

In tensorboard, you can monitor the losses computed on the training set and validation set (if you have one), which can help mitigate the risk of overfitting. For example, you might want to stop the training process if the reconstruction loss starts to increase.

In [ ]:
# optional: invoke TensorBoard with the %tensorboard magic command.

%load_ext tensorboard
%tensorboard --logdir lightning_logs

In [5]:
# train a LEOPARD

trainer.fit(my_leopard)

Missing logger folder: lightning_logs\trainNum_all\obsNum_0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                              | Type    | Params
--------------------------------------------------------------
0 | loss_F_reconstruction_noReduction | MSELoss | 0     
1 | loss_F_reconstruction             | MSELoss | 0     
2 | loss_F_adversarial                | MSELoss | 0     
3 | leopard                           | LEOPARD | 134 K 
--------------------------------------------------------------
134 K     Trainable params
0         Non-trainable params
134 K     Total params
0.538     Total estimated model params size (MB)
C:\Users\SiyuHan\anaconda3\envs\LEOPARD\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoa

## Step 6: Impute and export data

You can impute the missing data and writing them into a .csv file. If you have ground truth for "vB_t2_test.csv", you can also export percent bias of the imputed data.

In [6]:
# impute data
imputed_data = trainer.predict(my_leopard, my_leopard.test_dataloader())[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\SiyuHan\anaconda3\envs\LEOPARD\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:245: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


In [7]:
# create folder for output
output_dir = os.path.join(save_dir, name, "version_" + str(trainer.logger.version), "results") 

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# export imputed data
pd.DataFrame(imputed_data["generated_data"]).to_csv(
            os.path.join(output_dir, "imputedData_obs" + str(obsNum) + ".csv"), index=False
)

In [ ]:
# export percent bias (only when groundtruth is available)
pd.DataFrame(imputed_data["raw_percentBias"]).to_csv(
            os.path.join(output_dir, "PB_obs" + str(obsNum) + ".csv"), index=False
)

## End
This manual is prepared based on our analysis and has been tested on our benchmark datasets. 
Please let us know if you found any issues.